## Code Snippet 3.3 - The code samples from the SCM described in Example 3.2

$$
\begin{align}
C &:= N_C \\
E &:= 4 \cdot C + N_E
\end{align}
$$

In [1]:
set.seed(1)

# generates a sample from the distribution entailed by the SCM
C <- rnorm(300)
E <- 4*C + rnorm(300)
c(mean(E), var(E))

[1]  0.1236532 16.1386767

In [2]:
cor.test(C,E)$p.value

[1] 1.674893e-176

In [3]:
# generates a sample from the intervention distribution do(C:=2); this changes the distribution of E
# repeat 2, 300 times
C <- rep(2, times=300)
E <- 4*C + rnorm(300)

c(mean(E), var(E))

[1] 7.936917 1.187035

In [4]:
cor.test(C,E)$p.value

Warning message in cor(x, y):
“the standard deviation is zero”

[1] NA

In [5]:
# Generates a sample from the intervention distribution do(E:=N~); this breaks the dependence between C and E
C <- rnorm(300)
E <- rnorm(300)
cor.test(C,E)$p.value

[1] 0.2114492

## Code Snippet 4.14

> On veut savoir dans quel sens on met la flèche dans un simple diagramme X -> Y ou X <- Y  
> On va effectuer un test d'hypothèse d'indépendance

In [6]:
library(dHSIC)
library(mgcv)

# generate data set
set.seed(1)
X <- rnorm(200)
Y <- X^3 + rnorm(200)

# fit models
modelforw <- gam(Y ~ s(X))
modelbackw <- gam(X ~ s(Y))

# Independence tests
dhsic.test(modelforw$residuals, X)$p.value

Loading required package: nlme
This is mgcv 1.8-28. For overview type 'help("mgcv-package")'.


[1] 0.7712288

In [7]:
dhsic.test(modelbackw$residuals, Y)$p.value

[1] 0.006993007

In [8]:
# computing likelihoods
- log(var(X)) - log(var(modelforw$residuals))

[1] 0.1420063

In [9]:
- log(var(modelbackw$residuals)) - log(var(Y))

[1] -1.014013

## Code Snippet 6.4


In [10]:
# generate a sample from the distribution entailed by the SCM
set.seed(1)
X3 <- runif(100) - 0.5
X1 <- 2*X3 + rnorm(100)
X2 <- (0.5*X1)^2 + rnorm(100)^2
X4 <- X2 + 2*sin(X3 + rnorm(100))

In [11]:
c(mean(X4), var(X4))

[1] 1.601906 4.107679

## Code Snippet 6.9

We perform an intervention on `code Snippet 6.4` with $do(X_2:=3)$

In [12]:
# generate a sample from the intervention distribution
set.seed(1)
X3 <- runif(100) - 0.5
X1 <- 2*X3 + rnorm(100)
X2 <- rep(3, 100)
X4 <- X2 + 2*sin(X3 + rnorm(100))

In [13]:
c(mean(X4), var(X4))

[1] 3.047535 1.718381

## Code Snippet 6.43


<img src='img.png' width=500px><img src='img2.png' width=250px>


On va mettre des coefficients à tous ces edges, le causal effect qu'on veut recover est +2 (see below)
On va dire que (-2) x (-1) = 2

> Mais en pratique on ne connait que les données et la structure causale

In [14]:
n <- 500

# generate a sample from the distr. ent. by the SCM
set.seed(1)
C <- rnorm(n)
A <- 0.8*rnorm(n)
K <- A + 0.1*rnorm(n)
X <- C - 2*A + 0.2*rnorm(n) 
F <- 3*X + 0.8*rnorm(n)
D <- -2*X + 0.5*rnorm(n)
G <- D + 0.5*rnorm(n)
Y <- 2*K - D + 0.2*rnorm(n)
H <- 0.5*Y + 0.1*rnorm(n)

In [19]:
# On voit que la prediction est fortement biaisé (// au 2 qu'on voulait)
# Car on a le backdoor path
lm(y=Y,X=X)$coefficients

ERROR: Error in terms.formula(formula, data = data): argument is not a valid model


In [16]:
lm.fit(Y ~ X)

ERROR: Error in lm.fit(Y ~ X): 'x' must be a matrix


In [31]:
lm(Y~X+K)$coefficients


(Intercept)           X           K 
-0.02342617  2.01335136  1.99520944

In [32]:
lm(Y~X+F+C+K)$coefficients


(Intercept)           X           F           C           K 
-0.02406900  1.98769241 -0.02641737  0.11526356  1.78759472

In [40]:
# On a un biaised estimate car on a mis trop de variable
lm(Y~X+F+C+K+H)$coefficients

(Intercept)            X            F            C            K            H 
-0.001184594  0.118040620  0.009319489  0.040111922  0.116977788  1.822644767

## Code Snippet 7.11


In [45]:
library(InvariantCausalPrediction)

# generate data from two environments
env <- c(rep(1, 400), rep(2, 700))
n <- length(env)
set.seed(1)
X1 <- rnorm(n)
X2 <- 1*X1 + c(rep(0.1,400), rep(1.0,700))*rnorm(n)
Y <- -0.7*X1 + 0.6*X2 + 0.1*rnorm(n)
X3 <- c(rep(-2,400),rep(-1,700))*Y + 2.5*X2 + 0.1*rnorm(n)

summary(lm(Y~-1+X1+X2+X3))

Loading required package: glmnet
Loading required package: Matrix
Loading required package: foreach
Loaded glmnet 2.0-16

Loading required package: mboost
Loading required package: parallel
Loading required package: stabs
This is mboost 2.9-1. See ‘package?mboost’ and ‘news(package  = "mboost")’
for a complete list of changes.




Call:
lm(formula = Y ~ -1 + X1 + X2 + X3)

Residuals:
      Min        1Q    Median        3Q       Max 
-0.200513 -0.043141  0.003154  0.048571  0.236081 

Coefficients:
    Estimate Std. Error t value Pr(>|t|)    
X1 -0.396212   0.008667  -45.71   <2e-16 ***
X2  1.381497   0.021377   64.63   <2e-16 ***
X3 -0.410647   0.011152  -36.82   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.06938 on 1097 degrees of freedom
Multiple R-squared:  0.9831,	Adjusted R-squared:  0.983 
F-statistic: 2.127e+04 on 3 and 1097 DF,  p-value: < 2.2e-16


In [46]:
ICP(cbind(X1,X2,X3),Y,env)


 *** 25% complete: tested 2 of 8 sets of variables 
 accepted set of variables 1,2


 Invariant Linear Causal Regression at level 0.01 (including multiplicity correction for the number of variables)
 Variables: X1, X2 show a significant causal effect
 
     LOWER BOUND  UPPER BOUND  MAXIMIN EFFECT  P-VALUE    
X1        -0.71        -0.68           -0.68  1.9e-06 ***
X2         0.59         0.61            0.59   0.0046 ** 
X3         0.00         0.00            0.00   0.1486    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

